In [1]:
import numpy as np
from numpy.random import randint
from numpy.random import choice
from collections import Counter
from random import uniform

In [144]:
def FindAllKmers(dna_string,k):
  kmers_list = []
  i = 0
  while i + k - 1 <= len(dna_string) - 1:
    kmers_list.append(dna_string[i:i+k])
    i = i + 1
  return kmers_list

In [145]:
def SelectRandomKmers(dna,k,t):
  random_kmers = []
  for dna_string in dna:
    random_kmers.append(FindAllKmers(dna_string, k)[randint(0, len(dna_string) - k + 1)]) #upper bound not included
  return random_kmers

In [146]:
def NucleotideIndex(nucleotide):
  if nucleotide == 'A':
    return 0
  elif nucleotide == 'C':
    return 1
  elif nucleotide == 'G':
    return 2
  else:
    return 3

In [147]:
def IndexNucleotide(index):
  if index == 0:
    return 'A'
  elif index == 1:
    return 'C'
  elif index == 2:
    return 'G'
  else:
    return 'T'

In [148]:
def KmerProbability(profile,kmer):
  probability = 1
  for i in range(len(kmer)):
    probability = probability * profile[NucleotideIndex(kmer[i])][i]
  return probability

In [149]:
def Consensus(profile,k):
  consensus = str()
  for i in range(k):
    consensus = consensus + IndexNucleotide(np.argmax(profile[:,i]))
  return consensus

In [150]:
def HammingDistance(first_string,second_string):
  hamming_distance = 0
  for i in range(min(len(first_string),len(second_string))):
    if first_string[i] != second_string[i]:
      hamming_distance = hamming_distance + 1
  return hamming_distance + abs(len(first_string) - len(second_string))

In [167]:
def DnaToArray(dna):
  dna_array = np.zeros((len(dna),len(dna[0])), dtype='str')
  for i in range(len(dna)):
    dna_array[i,:] = np.asarray(list(dna[i]), dtype='str')
  return dna_array

In [311]:
def GenerateProfile(motifs,k):
  profile = np.zeros((4,k))
  motifs_array = DnaToArray(motifs)
  for i in range(k):
    nucleotide_frequency_dict = Counter(motifs_array[:,i])
    for nucleotide_index in range(4):
      profile[nucleotide_index][i] = nucleotide_frequency_dict[IndexNucleotide(nucleotide_index)]
  profile = profile / len(motifs)
  return profile

In [313]:
def Score(motifs,k):
  score = 0
  profile = GenerateProfile(motifs,k)
  consensus = Consensus(profile,k)
  for motif in motifs:
    score = score + HammingDistance(consensus,motif)
  return score

In [330]:
def LaplacesRuleOfSucccession(profile):
  return profile + 1

In [248]:
from random import choices

In [255]:
def ProfileRandomlyGeneratedKmer(dna_string,profile,k):
  kmers = FindAllKmers(dna_string,k)
  kmers_probabilities = []
  for kmer in kmers:
    kmers_probabilities.append(KmerProbability(profile,kmer))
  kmers_probabilities_sum = sum(kmers_probabilities)
  for i in range(len(kmers_probabilities)):
    kmers_probabilities[i] = kmers_probabilities[i] / kmers_probabilities_sum
  return kmers[choices(list(range(len(dna_string)-k+1)),kmers_probabilities)[0]]

In [314]:
def GibbsSampler(dna,k,t,N):
  motifs = SelectRandomKmers(dna,k,t)
  best_motifs = motifs
  best_score = Score(best_motifs,k)
  for j in range(N):
    i = randint(t)
    motifs.pop(i)
    profile = LaplacesRuleOfSucccession(GenerateProfile(motifs,k))
    motif_i = ProfileRandomlyGeneratedKmer(dna[i],profile,k)
    motifs.insert(i,motif_i)
    if Score(motifs,k) < Score(best_motifs,k):
      best_motifs = motifs
      best_score = Score(motifs,k)
  return best_score,best_motifs

In [315]:
from math import inf

In [316]:
def RunGibbsSampler(dna,k,t,N):
  best_motifs = []
  best_score = inf
  for i in range(20):
    score,motifs = GibbsSampler(dna,k,t,N)
    if score < best_score:
      best_motifs = motifs
      best_score = score
  return best_motifs

We now define a Profile-randomly generated k-mer in a string Text. For each k-mer Pattern in Text, compute the probability Pr(Pattern|Profile), resulting in n = |Text| - k + 1 probabilities (p1, ... , pn). These probabilities do not necessarily sum to 1, but we can still form the random number generator RANDOM(p1, ... , pn) based on them. GIBBSSAMPLER uses this random number generator to select a Profile-randomly generated k-mer at each step: if the die rolls the number i, then we define the Profile-randomly generated k-mer as the i-th k-mer in Text. 

statistical maxim called Cromwell’s rule, which states that we
should not use probabilities of 0 or 1 unless we are talking about logical statements that can only be true or false (statement jedino može biti istina ili laž, tj. imamo samo 2 moguća ishoda). In other words, we should allow a small probability for extremely unlikely events, such as “this book was written by aliens” or “the sun will not rise tomorrow”. --> dakle, trebali bi odrediti kardinalni broj skupa elementarnih događaja tako da znamo kada možemo koristiti vjerojatnosti 0 i 1, a kada ne (P(Ω) = 1, kod uniformnog vjerojatnosnog modela svi elementarni događaju su jednako vjerojatni, ako ih ima n onda je vjerojatnost svakog od njih 1/n, dakle nijedna vjerojatnost nije 0, ako slučajni pokus sa 2 moguća ishoda (uspjeh ili neuspjeh) modeliramo Bernoullijevom slučajnom varijablom onda je vjerojatnost P(X=1)=p, P(X=0)=1-p=q, dakle ako je p = 1 onda slučajni pokus ima samo jedan ishod i nema smisla govoriti o vjerojatnosti 0, ako je p = 0 onda slučajni pokus ima samo jedan ishod i ima smisla govoriti o vjerojatnosti 0, elementarni događaji su svi mogući, a nedjeljivi, ishodi slučajnog pokusa, dakle treba u obzir uzeti sve elementarne događaje)

**Cromwell’s rule is relevant to the calculation of the probability of a string based on a profile matrix**. For example, consider the following Profile:

A: .2 .2 .0 .0 .0 .0 .9 .1 .1 .1 .3 .0

C: .1 .6 .0 .0 .0 .0 .0 .4 .1 .2 .4 .6

G: .0 .0  1  1 .9 .9 .1 .0 .0 .0 .0 .0

T: .7 .2 .0 .0 .1 .1 .0 .5 .8 .7 .3 .4

Pr(TCGTGGATTTCC|Profile) = .7 · .6 · 1 · .0 · .9 · .9 · .9 · .5 · .8 · .7 · .4 · .6 = 0

The fourth symbol of TCGTGGATTTCC causes Pr(TCGTGGATTTCC|Profile) to equal zero (event with non-zero probability didn't occur, its observed frequency is zero, setting its probability to zero represents an inaccurate oversimplification that may cause problems --> profile matricu nismo generirali sa kmerima koji su imali nukleotid T na 4. poziciji, vjerojatnost da je T na 4. poziciji nije 0). As a result, the entire string is assigned a zero probability, even though TCGTGGATTTCC differs from the consensus string at only one position (inaccurate oversimplification that caused problems). For that matter, TCGTGGATTTCC has the same low probability as AAATCTTGGAA, which is very different from the consensus string (inaccurate oversimplification that caused problems). In order to improve this unfair scoring, bioinformaticians often substitute zeroes with small numbers called pseudocounts. In the case of
motifs, pseudocounts often amount to adding 1 (or some other small number) to each element of COUNT(Motifs).

In [259]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA', 'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG', 'TAGTACCGAGACCGAAAGAAGTATACAGGCGT', 'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC', 'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']

In [260]:
k = 8

In [261]:
t = 5

In [262]:
N = 100

In [266]:
RunGibbsSampler(dna,k,t,N)

['GTGTTCAG', 'AAGTGCCA', 'GTATACAG', 'AGGTGCAC', 'ATGTTGGC']

In [324]:
with open('/content/rosalind_ba2g.txt') as task_file:
  task_arguments = [line.rstrip() for line in task_file]

In [325]:
k = int(task_arguments[0][0:2])

In [326]:
t = int(task_arguments[0][3:5])

In [327]:
N = int(task_arguments[0][6:len(task_arguments[0])])

In [328]:
dna = task_arguments[1:len(task_arguments)]

In [329]:
f = open("task_result.txt", "w")
for solution in RunGibbsSampler(dna,k,t,N):
  f.write(solution + '\n')
f.close()